# Import libraries

In [8]:
import os
import sys
import tweepy
import requests
from dotenv import load_dotenv

sys.path.append('../../')
import config

load_dotenv(os.path.join(r'C:\Users\DCandelero\Documents\MBA - USP(Data Analytics)\MBA_TCC', '.env')) 

True

# Set twitter access

In [9]:
consumer_key = os.environ["API_KEY"]
consumer_secret = os.environ["API_KEY_SECRET"]
access_token = os.environ["ACCESS_TOKEN"]
access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]

auth = tweepy.OAuth1UserHandler(
  consumer_key, 
  consumer_secret, 
  access_token, 
  access_token_secret
)

api = tweepy.API(auth)

# Create functions to read and save texts and images

In [140]:
def save_tweet_text(tweet:object) -> None:
    tweet_timestamp = str(tweet.created_at.strftime("%Y-%m-%d %H-%M-%S"))
    tweet_id = str(tweet.id)
    txt_file_name = os.path.join(config.DATA_PATH_RAW_TEXTS, '{}_{}.txt'.format(tweet_timestamp, tweet_id))
    
    with open(txt_file_name, 'w', encoding="utf-8") as txt_file:
        try:
            txt_file.write(tweet.full_text)
            # txt_file.write(tweet.retweeted_status.full_text)
            txt_file.write("\n=====\n")
        except AttributeError:
            print('Unable to read text from tweet {}'.format(tweet.id))
            print("=====")

    return None

def DownloadFile(url:str, path_to_save:str) -> None:
    response = requests.get(url)

    if response.status_code == 200:
        with open(path_to_save, 'wb') as f:
                f.write(response.content)
    return None


def save_tweet_image(tweet:object) -> None:
    tweet_timestamp = str(tweet.created_at.strftime("%Y-%m-%d %H-%M-%S"))
    tweet_id = str(tweet.id)

    try:
        for media in tweet.entities.get("media",[{}]):
            #checks if there is any media-entity
            if media.get("type",None) == "photo":
                tweet_media_id = str(media['id'])
                filename = os.path.join(config.DATA_PATH_RAW_IMAGES, '{}_{}_{}.png'.format(tweet_timestamp, tweet_id, tweet_media_id))
                DownloadFile(media["media_url"], filename)
    except AttributeError:
        print('Unable to read medias from tweet {}'.format(tweet_id))
        print("=====")

# Pull tweets

In [145]:
tweets = api.search_tweets("violence against women", tweet_mode="extended", count=100)

# Read tweets
for tweet in tweets:
    save_tweet_text(tweet)

    save_tweet_image(tweet)

In [146]:
vars(tweets[0]).keys()

dict_keys(['_api', '_json', 'created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'metadata', 'source', 'source_url', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'author', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])